In [37]:
import os

import pandas as pd
%matplotlib inline
%matplotlib notebook
from CheXpert2 import names
import cv2
from CheXpert2.dataloaders.CXRLoader import CXRLoader
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [38]:
img_dir="/mnt/c/Users/bejo2361/PycharmProjects/IA-MED_IMG"
dataset = CXRLoader(
            split="Train",
            img_dir=img_dir,
            img_size=224,
            prob=[0,1,0,1,0,0],
            intensity=0,
            label_smoothing=0,
            channels=1,
            use_frontal=False,
            datasets=["CIUSSS"]
        )
i=0

/mnt/c/Users/bejo2361/PycharmProjects/IA-MED_IMG/CheXpert2/dataloaders/CXRLoader.py:179: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data=data.groupby("Exam ID").mean().round(0)


In [39]:
images,label,idx = dataset[i]
print(images.shape)
frontal,lateral =   images[0,:,:].numpy().astype(np.uint8),images[1,:,:].numpy().astype(np.uint8)
#frontal,lateral = cv2.cvtColor(frontal[:,:,None],cv2.COLOR_GRAY2RGB),cv2.cvtColor(lateral[:,:,None],cv2.COLOR_GRAY2RGB)
print(np.max(frontal))
print(np.max(lateral))
frontal_image = Image.fromarray(frontal)
lateral_image = Image.fromarray(lateral)
frontal_image.show()
lateral_image.show()
i+=1

[ WARN:0@92323.405] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/mnt/c/Users/bejo2361/PycharmProjects/IA-MED_IMG/data/public_data/Images_CIUSSS/1000013805945086/1000013805945086_view0002.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [41]:
data = np.concatenate([dataset.count.reshape((1,18)),dataset.count.reshape((1,18))*len(dataset)])
data = pd.DataFrame(data,columns=names,index=["Ratio","Count"])
print(data)

            Opacity          Air        Liquid  Cardiomegaly  Lung Lesion  \
Ratio      0.235656     0.034475      0.174541      0.079604     0.015898   
Count  42469.000000  6213.000000  31455.000000  14346.000000  2865.000000   

         Emphysema        Edema  Consolidation   Atelectasis  Pneumothorax  \
Ratio     0.014693     0.009844        0.01734      0.115672       0.01102   
Count  2648.000000  1774.000000     3125.00000  20846.000000    1986.00000   

       Pleural Effusion     Fracture       Hernia  Infiltration         Mass  \
Ratio          0.118264     0.015298     0.011447      0.064306     0.015875   
Count      21313.000000  2757.000000  2063.000000  11589.000000  2861.000000   

            Nodule  Pneumo other   No Finding  
Ratio     0.016092      0.004339      0.45774  
Count  2900.000000    782.000000  82492.00000  
